In [9]:
import pandas as pd
import numpy as np
import math
import random

# Importing dataset

In [11]:
datasets = [
    "data/opsahl-ucsocial/out.opsahl-ucsocial",
    "data/emails/email-Eu-core-temporal_processed.txt",
    "data/soc-sign-bitcoinalpha/out.soc-sign-bitcoinalpha",
    "data/soc-sign-bitcoinotc/out.soc-sign-bitcoinotc",
    "data/munmun_digg_reply/out.munmun_digg_reply",
    "data/sx-mathoverflow/out.sx-mathoverflow"
]
skiprows = [[0, 1], [], [0], [0], [0], [0]]
current = 0 # 0 1 2 3 4 5
graph = pd.read_csv(
    datasets[current],
    names=["_from", "_to", "_weight", "_timestamp"],
    sep=" |\t",
    engine ='python',
    skiprows=skiprows[current]
)
# print(graph.dtypes)
# print(graph.head())

# Helpful tools

In [49]:
def write_results(number_of_task: int, lines: list[str]):
    """Writes results to .txt files.
    Args:
        number_of_task (int): 1 or 2.
        lines (list[str]): Lines to write.
    """
    with open(f"results/task_{number_of_task}/dataset_{current}.txt", "w") as f:
        f.writelines(line + '\n' for line in lines)

In [12]:
V = np.unique(graph["_from"]._append(graph["_to"])).astype(int)
V.sort()
n = V.size
volume = graph["_timestamp"].size
print(f"{n = }, {volume = }")

n = 1899, volume = 59835


# Chapter 1

## Preparing static graph

In [13]:
graph_static = dict.fromkeys(V)
for u in V:
    graph_static[u] = set()
for _index, _from, _to, _weight, _timestamp in graph.itertuples():
    if _from == _to:
        continue
    if not _to in V:
        print(_from, _to)
    graph_static[_from].add(_to)
    graph_static[_to].add(_from)


## Task 1.1

In [31]:
E_count = 0
for u in graph_static.keys():
    E_count += len(graph_static[u])
E_count //= 2
#print(E_count)

density = E_count * 2 / (n * (n - 1))
#print(density)

In [33]:
V_to_visit = set(V)
connectivity_components = []
while(V_to_visit):
    V_seen = set()
    queue = []
    for u in V_to_visit:
        queue.append(u)
        V_seen.add(u)
        break
    while queue:
        u = queue.pop()
        u_adjacent_to_visit = graph_static[u].difference(V_seen)
        for v in u_adjacent_to_visit:
            V_seen.add(v)
            queue.append(v)
    V_to_visit = V_to_visit.difference(V_seen)
    connectivity_components.append(V_seen)

sizes = list(map(lambda x: len(x), connectivity_components))
max_component_size = max(sizes)
max_connectivity_component_index = sizes.index(max_component_size)
proportion = max_component_size / len(V)
#print(f"{max_val = }, {max_connectivity_component_index = }, {proportion = }")


In [ ]:
# 1.1
print("|V| = %i, |E| = %i, p = %f, number of components = %i, max component size = %i, max component proportion = %f"
      % (n, E_count, density, len(connectivity_components), max_component_size, proportion))


## Task 1.2

In [34]:
component = list(connectivity_components[max_connectivity_component_index])
distances = [0] * (n + 2)
diameter = 0
radius = n + 1
n_limit = 10000
if n < n_limit:
    for start in component:
        distances_tmp = dict.fromkeys(V, n + 1)
        V_visited = set()
        queue = [(start, 0)]
        queued = set([start])
        depth = 0
        u = start
        while queued:
            (u, depth) = queue.pop(0)
            queued.remove(u)
            V_visited.add(u)
            u_adjacent_to_visit = graph_static[u].difference(V_visited)
            for v in u_adjacent_to_visit:
                distances_tmp[v] = min(distances_tmp[v], depth + 1)
                if v not in queued:
                    queue.append((v, depth + 1))
                    queued.add(v)
        for u in V:
            if u > start:
                distances[distances_tmp[u]] += 1
        diameter = max(diameter, depth)
        radius = min(radius, depth)
        if not start % 10:
            #print(start, "/", n)
            pass
all_dist = 0
for i in range(diameter + 1):
    all_dist += distances[i]
print(all_dist, "=", max_component_size * (max_component_size - 1) // 2,
      all_dist == max_component_size * (max_component_size - 1) // 2)


1790778 = 1790778 True


In [35]:
#percentile_90 = np.percentile(all_distances, 90)
percentile_90_ind = int(0.9 * all_dist)
percentile_90 = 0
ind_tmp = 0
for i in range(diameter + 1):
    ind_tmp += distances[i]
    if ind_tmp >= percentile_90_ind:
        percentile_90 = i
        break

In [37]:
def calculate_matrix(vertices):

    vertices_list = list(vertices)

    distances = []
    distance_matrix = dict()
    for u in vertices_list:
        distance_matrix[u] = dict()
        for v in vertices_list:
            if u != v:
                distance_matrix[u][v] = n + 1

    for start in vertices_list:
        V_to_calculate = set(vertices)
        V_to_calculate.discard(start)
        V_visited = set()
        queue = [(start, 0)]
        queued = set([start])
        max_depth = 0
        while queued and V_to_calculate:
            u, depth = queue.pop(0)
            max_depth = max(max_depth, depth)
            queued.discard(u)
            V_visited.add(u)
            u_adjacent_to_visit = graph_static[u].difference(V_visited)
            for v in u_adjacent_to_visit:
                if v in V_to_calculate:
                    distance = distance_matrix[start][v]
                    if depth + 1 < distance:
                        distance_matrix[start][v] = depth + 1
                        distance_matrix[v][start] = depth + 1
                        V_to_calculate.discard(v)
                if v not in queued:
                    queue.append((v, depth + 1))
                    queued.add(v)
    
    for u in vertices_list:
        for v in vertices_list:
            if u > v:
                distances.append(distance_matrix[u][v])
    eccentricities = dict()
    for u in distance_matrix:
        eccentricities[u] = max(distance_matrix[u].values())

    return (distance_matrix, eccentricities, distances)


In [38]:

component = list(connectivity_components[max_connectivity_component_index])
diameter_from_random_500 = 0
radius_from_random_500 = 0
percentile_90_from_random_500 = 0
diameter_from_random_1000 = 0
radius_from_random_1000 = 0
percentile_90_from_random_1000 = 0
if n >= 500:
    random_500_vertices = sorted(random.sample(component, 500))
    random_500_matrix, random_500_eccentricities, random_500_distances = calculate_matrix(
        random_500_vertices)
    diameter_from_random_500 = max(random_500_eccentricities.values())
    radius_from_random_500 = min(random_500_eccentricities.values())
    percentile_90_from_random_500 = np.percentile(random_500_distances, 90)
    print(f"{diameter_from_random_500 = }, {radius_from_random_500 = }, {percentile_90_from_random_500 = }")
if n >= 1000:
    random_1000_vertices = sorted(random.sample(component, 1000))
    random_1000_matrix, random_1000_eccentricities, random_1000_distances = calculate_matrix(
        random_1000_vertices)
    diameter_from_random_1000 = max(random_1000_eccentricities.values())
    radius_from_random_1000 = min(random_1000_eccentricities.values())
    percentile_90_from_random_1000 = np.percentile(random_1000_distances, 90)
    print(f"{diameter_from_random_1000 = }, {radius_from_random_1000 = }, {percentile_90_from_random_1000 = }")

                    

diameter_from_random_500 = 8, radius_from_random_500 = 4, percentile_90_from_random_500 = 4.0
diameter_from_random_1000 = 8, radius_from_random_1000 = 4, percentile_90_from_random_1000 = 4.0


In [39]:
def snowball(limit):
    vertices = {component[0], component[1]}
    while len(vertices) < limit:
        for v in vertices:
            if len(vertices) < limit:
                vertices = vertices.union(graph_static[v])
    return sorted(list(vertices))


diameter_from_snowball_500 = 0
radius_from_snowball_500 = 0
percentile_90_from_snowball_500 = 0
diameter_from_snowball_1000 = 0
radius_from_snowball_1000 = 0
percentile_90_from_snowball_1000 = 0
if n >= 500:
    snowball_500_vertices = snowball(500)
    snowball_500_matrix, snowball_500_eccentricities, snowball_500_distances = calculate_matrix(
        snowball_500_vertices)
    diameter_from_snowball_500 = max(snowball_500_eccentricities.values())
    radius_from_snowball_500 = min(snowball_500_eccentricities.values())
    percentile_90_from_snowball_500 = np.percentile(snowball_500_distances, 90)
    print(f"{diameter_from_snowball_500 = }, {radius_from_snowball_500 = }, {percentile_90_from_snowball_500 = }")

if n >= 1000:
    snowball_1000_vertices = snowball(1000)
    snowball_1000_matrix, snowball_1000_eccentricities, snowball_1000_distances = calculate_matrix(
        snowball_1000_vertices)
    diameter_from_snowball_1000 = max(snowball_1000_eccentricities.values())
    radius_from_snowball_1000 = min(snowball_1000_eccentricities.values())
    percentile_90_from_snowball_1000 = np.percentile(snowball_1000_distances, 90)
    print(f"{diameter_from_snowball_1000 = }, {radius_from_snowball_1000 = }, {percentile_90_from_snowball_1000 = }")


diameter_from_snowball_500 = 4, radius_from_snowball_500 = 3, percentile_90_from_snowball_500 = 3.0
diameter_from_snowball_1000 = 5, radius_from_snowball_1000 = 3, percentile_90_from_snowball_1000 = 3.0


In [40]:
# 1.2
print("diameter = %i, raduis = %i, percentile_90 = %i" 
      % (diameter, radius, percentile_90))

diameter = 8, raduis = 4, percentile_90 = 4


## Task 1.3

In [ ]:
component = list(connectivity_components[max_connectivity_component_index])

Cl = dict()
for u in component:
    u_neighbors = graph_static[u]

    if len(u_neighbors) < 2:
        Cl[u] = 0
        continue

    Lu_doubled = 0
    for neighbor in u_neighbors:
        Lu_doubled += len(graph_static[neighbor].intersection(u_neighbors))
    Cl[u] = Lu_doubled / (len(u_neighbors) * (len(u_neighbors) - 1))

Cl_average = sum(Cl.values()) / len(Cl.values())

In [ ]:
#1.3
print("Cl_average = %f" % (Cl_average))

## Task 1.4

In [ ]:
R1 = 0
R2 = 0
R3 = 0
Re = 0
for u in V:
    ku = len(graph_static[u])
    R1 += ku
    R2 += ku**2
    R3 += ku**3
    for v in graph_static[u]:
        kv = len(graph_static[v])
        Re += ku * kv
degree_associativity = (Re * R1 - R2**2) / (R3 * R1 - R2**2)

In [ ]:
print("Degree associativity = %f" % (degree_associativity))

# Chapter 2

In [17]:
t_min = graph["_timestamp"].min()
t_max = graph["_timestamp"].max()
s = 2.0 / 3.0
t_s = t_min + (t_max - t_min) * s
l = 0.2

t_matrix = dict.fromkeys(V)
for u in V:
    t_matrix[u] = dict()

for _index, _from, _to, _weight, _timestamp in graph.itertuples():
    if _from == _to or _timestamp > t_s:
        continue
    
    if t_matrix[_from].get(_to) is not None: 
        t_matrix[_from][_to].append(_timestamp)
    else:
        t_matrix[_from][_to] = [_timestamp]
    
    if t_matrix[_to].get(_from) is not None: 
        t_matrix[_to][_from].append(_timestamp)
    else:
        t_matrix[_to][_from] = [_timestamp]
    

## Static topological features

In [18]:
def get_static_topological_features(u, v):
    gamma_u = set(t_matrix[u].keys())
    gamma_v = set(t_matrix[v].keys())
    
    intersection_u_v = gamma_u.intersection(gamma_v)
    
    CN_static = len(intersection_u_v)
    JC_static = CN_static / len(gamma_u.union(gamma_v))
    PA_static = len(gamma_u) * len(gamma_v)
    AA_static = sum(
        [1.0 / np.log(len(set(t_matrix[z].keys()))) for z in intersection_u_v]
    )
    return CN_static, AA_static, JC_static, PA_static

## Node activity features

### Step 1: Temporal weighting

In [19]:
def get_temporal_weighting(l, t):
    time_var = (t - t_min) / (t_s - t_min)
    w_linear = l + (1 - l) * time_var
    w_exponential = l + (1 - l) * (np.exp(3 * time_var) - 1) / (np.exp(3) - 1)
    w_square_root = l + (1 - l) * np.sqrt(time_var)
    return [w_linear, w_exponential, w_square_root]

for vt in t_matrix.values():
    for v, times in vt.items():
        for i, t in enumerate(times):
            vt[v][i] = get_temporal_weighting(l, t)
    

### Step 2: Aggregation of node activity

In [20]:
class AggregationOfNodeActivity:  
    @staticmethod
    def zeroth_quantile(weights):
        return np.quantile(weights, 0)
    
    @staticmethod
    def first_quantile(weights):
        return np.quantile(weights, 0.25)
    
    @staticmethod
    def second_quantile(weights):
        return np.quantile(weights, 0.50)
    
    @staticmethod
    def third_quantile(weights):
        return np.quantile(weights, 0.75)
    
    @staticmethod
    def fourth_quantile(weights):
        return np.quantile(weights, 1)
    
    @staticmethod
    def get_sum(weights):
        return sum(weights)
    
    @staticmethod
    def get_mean(weights):
        return np.mean(weights)
    
    @staticmethod
    def aggregate(weights):
        return [
            AggregationOfNodeActivity.zeroth_quantile(weights),
            AggregationOfNodeActivity.first_quantile(weights),
            AggregationOfNodeActivity.second_quantile(weights),
            AggregationOfNodeActivity.third_quantile(weights),
            AggregationOfNodeActivity.fourth_quantile(weights),
            AggregationOfNodeActivity.get_sum(weights),
            AggregationOfNodeActivity.get_mean(weights)
        ]


node_activity_feature_graph = dict()
for u in V:
    node_activity_feature_graph[u] = list()

for u, vt in t_matrix.items():
    if len(vt) == 0:
        continue
    linears = []
    exponentials = []
    squares = []
    for v, weights in vt.items():
        if not linears:
            linears = [w[0] for w in weights]
            exponentials = [w[1] for w in weights]
            squares = [w[2] for w in weights]
            continue
        linears.extend([w[0] for w in weights])
        exponentials.extend([w[1] for w in weights])
        squares.extend([w[2] for w in weights])
    for weights in (linears, exponentials, squares):
        node_activity_feature_graph[u].extend(
            AggregationOfNodeActivity.aggregate(weights)
        )

### Step 3: Combining node activity

In [21]:
class CombiningNodeActivity:
    @staticmethod
    def get_sum(a, b):
        return a + b
    
    @staticmethod
    def get_absolute_differrence(a, b):
        return abs(a - b)
    
    @staticmethod
    def get_minimum(a, b):
        return min(a, b)
    
    @staticmethod
    def get_maximum(a, b):
        return max(a, b)
    
    @staticmethod
    def combine(u_list, v_list):
        feature = []
        for a, b in zip(u_list, v_list):
            feature.append(CombiningNodeActivity.get_sum(a, b))
            feature.append(CombiningNodeActivity.get_absolute_differrence(a, b))
            feature.append(CombiningNodeActivity.get_minimum(a, b))
            feature.append(CombiningNodeActivity.get_maximum(a, b))
        return feature



In [23]:
pairs = set()

for u in V:
    u_adj = set(t_matrix[u].keys())
    for v in V:
        if u >= v or v in u_adj:
            continue
        v_adj = set(t_matrix[v].keys())
        if v_adj.intersection(u_adj):
            pairs.add((u, v))


In [24]:
connecting = list()
non_connecting = list()

for u, v in pairs:
    if v in graph_static[u]:
        connecting.append((u, v))
    else:
        non_connecting.append((u, v))

print(len(connecting), len(non_connecting))

220 334060


In [25]:
m = 10000
x_y = random.choices(connecting, k=m)
x_n = random.choices(non_connecting, k=m)

# all = train + test

x_pairs = list(x_y)
x_pairs.extend(x_n)

y_all = [1] * m
y_all.extend([0] * m)

features_temporal = dict.fromkeys(list(set(x_y).union(set(x_n))))
features_static = dict.fromkeys(list(set(x_y).union(set(x_n))))

for u, v in features_temporal.keys():
    features_static[(u, v)] = get_static_topological_features(u, v)
    features_temporal[(u, v)] = CombiningNodeActivity.combine(
        node_activity_feature_graph[u], node_activity_feature_graph[v]
    )

x_all_static = []
x_all_temporal = []

for pair in x_pairs:
    x_all_static.append(features_static[pair])
    x_all_temporal.append(features_temporal[pair])


y_train = []
y_test = []
x_train_static = []
x_test_static = []
x_train_temporal = []
x_test_temporal = []
x_pairs_train = []
x_pairs_test = []

for i in range(2 * m):
    if random.random() > 0.75:
        y_test.append(y_all[i])
        x_test_static.append(x_all_static[i])
        x_test_temporal.append(x_all_temporal[i])
        x_pairs_test.append(x_pairs[i])
    else:
        y_train.append(y_all[i])
        x_train_static.append(x_all_static[i])
        x_train_temporal.append(x_all_temporal[i])
        x_pairs_train.append(x_pairs[i])


In [27]:
from sklearn.linear_model import LogisticRegression
from matplotlib import pyplot as plt
from sklearn import metrics

In [28]:
model_temporal = LogisticRegression(max_iter=10000)
model_temporal.fit(x_train_temporal, y_train)
y_pred_temporal = model_temporal.predict(x_test_temporal)
print(metrics.accuracy_score(y_test, y_pred_temporal))

0.8255020080321285


In [29]:
model_static = LogisticRegression(max_iter=1000)
model_static.fit(x_train_static, y_train)
y_pred_static = model_static.predict(x_test_static)
print(metrics.accuracy_score(y_test, y_pred_static))

0.6853413654618474


# Writing results down

In [50]:
lines = [
    "Task 1.1",
    f"Число вершин: |V| = {n}",
    f"Число ребер: |E| = {E_count}",
    f"Плотность: p = {density}"
    f"Число компонент слабой связности: {len(connectivity_components)}",
    f"Доля вершин в максимальной по мощности компоненте слабой связности: {max_component_size}",
    "\nTask 1.2",
    f"Оценки для 500 вершин (из наибольшей КСС): {diameter_from_random_500 = }, {radius_from_random_500 = }, {percentile_90_from_random_500 = }",
    f"Оценки для 1000 вершин (из наибольшей КСС): {diameter_from_random_1000 = }, {radius_from_random_1000 = }, {percentile_90_from_random_1000 = }",
    f"Оценки для 500 вершин (снежный ком): {diameter_from_snowball_500 = }, {radius_from_snowball_500 = }, {percentile_90_from_snowball_500 = }",
    f"Оценки для 1000 вершин (снежный ком): {diameter_from_snowball_1000 = }, {radius_from_snowball_1000 = }, {percentile_90_from_snowball_1000 = }",
    "a" if True: else "b",
    
]
write_results(1, lines)

In [30]:
f = open("results/Dataset"+str(current)+".txt", "w")

f.write(f"{n = }, {volume = }")
f.write('\n')
f.write("|V| = %i, |E| = %i, p = %f, number of components = %i, max component size = %i, max component proportion = %f"
        % (n, E_count, density, len(connectivity_components), max_component_size, proportion))
f.write('\n')
# f.write("%i = %i, %s" %
#         (all_dist, max_component_size * (max_component_size - 1) // 2, all_dist == max_component_size * (max_component_size - 1) // 2))
# f.write('\n')
f.write(f"{diameter_from_random_500 = }, {radius_from_random_500 = }, {percentile_90_from_random_500 = }")
f.write('\n')
f.write(f"{diameter_from_random_1000 = }, {radius_from_random_1000 = }, {percentile_90_from_random_1000 = }")
f.write('\n')
f.write(f"{diameter_from_snowball_500 = }, {radius_from_snowball_500 = }, {percentile_90_from_snowball_500 = }")
f.write('\n')
f.write(f"{diameter_from_snowball_1000 = }, {radius_from_snowball_1000 = }, {percentile_90_from_snowball_1000 = }")
f.write('\n')
if n < n_limit:
    f.write("diameter = %i, raduis = %i, percentile_90 = %i" % (diameter, radius, percentile_90))
else:
    f.write("could not find diameter, raduis, percentile_90")
f.write('\n')
f.write("Cl_average = %f" % (Cl_average))
f.write('\n')
f.write("Degree associativity = %f" % (degree_associativity))
f.close()

NameError: name 'E_count' is not defined